# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [16]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-53573cab-fc5d-e8af-4423-124f958899fb)
GPU 1: NVIDIA GeForce RTX 3090 (UUID: GPU-c0e0a88f-c719-20b0-0c44-8b697299c276)
GPU 2: NVIDIA GeForce RTX 3090 (UUID: GPU-0aa8929e-1774-1dc0-5325-0dce1165c78c)
GPU 3: NVIDIA GeForce RTX 3090 (UUID: GPU-9f605028-04a0-7456-bed1-8e4e06d9d44a)
GPU 4: NVIDIA GeForce RTX 3090 (UUID: GPU-31d09e65-80fe-5f2a-c34a-17da33412ae0)
GPU 5: NVIDIA GeForce RTX 3090 (UUID: GPU-d1a545c6-1038-527d-ec65-40f66caf2807)
GPU 6: NVIDIA GeForce RTX 3090 (UUID: GPU-7db6b504-9087-7bac-ba63-f957f018e494)
GPU 7: NVIDIA GeForce RTX 3090 (UUID: GPU-65bf5d44-7fd3-2034-09cf-4db9a2746b2a)


### Setup the model

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import torch
import torch.nn as nn
import bitsandbytes as bnb
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of BloomForCausalLM were not initialized from the model checkpoint at bigscience/bloom-7b1 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Freezing the original weights


In [4]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


## SJMM Data

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
# pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_columns', None)
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"

# data_df = pd.read_csv(data_path)



# train_ratio = 0.9
# val_ratio = 0.1
# test_ratio = 0.1
from sklearn.model_selection import train_test_split
sjmm_data = pd.read_csv(data_path)
sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
# validation_sjmm_data, test_sjmm_data = train_test_split(test_sjmm_data, test_size=0.5, random_state=42)

In [8]:
# Create a new Dataset object
from datasets import Dataset, DatasetDict
train_sjmm_dataset = Dataset.from_dict(train_sjmm_data)

# Create a new DatasetDict object with the 'train' subset
train_sjmm_dataset_dict = DatasetDict({'train': train_sjmm_dataset})

In [9]:
train_sjmm_dataset_dict['train'][0]

{'input': '•  <SoftSkill_C>Hohes Kommunikations-</SoftSkill_C> und <SoftSkill>Verhandlungsgeschick</SoftSkill',
 'output': '<SoftSkill>Hohes Kommunikationsgeschick</SoftSkill> und <SoftSkill>Verhandlungsgeschick</SoftSkill>',
 'ind_skill': ['Hohes Kommunikationsgeschick', 'Verhandlungsgeschick']}

In [10]:
def merge_columns(example):
    example["prediction"] = example["input"] + " == " + str(example["ind_skill"])
    return example

train_sjmm_dataset_dict['train'] = train_sjmm_dataset_dict['train'].map(merge_columns)
train_sjmm_dataset_dict['train']["prediction"][:5]

Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

["•  <SoftSkill_C>Hohes Kommunikations-</SoftSkill_C> und <SoftSkill>Verhandlungsgeschick</SoftSkill == ['Hohes Kommunikationsgeschick', 'Verhandlungsgeschick']",
 " Bist du eine <SoftSkill_C>vielseitige</SoftSkill_C>, <SoftSkill_C>motivierte</SoftSkill_C> und <SoftSkill>lernorientierte Person</SoftSkill>, die <SoftSkill>Freude an der aktiven Gestaltung und Entwicklung anspruchsvoller Lösungen v.a. im Bereich der Digitalisierung hat</SoftSkill>? == ['vielseitige Person', 'motivierte Person', 'lernorientierte Person', 'Freude an der aktiven Gestaltung und Entwicklung anspruchsvoller Lösungen v.a. im Bereich der Digitalisierung hat']",
 " •  <SoftSkill>Überzeugender Auftritt</SoftSkill>; <SoftSkill_C>dynamische</SoftSkill_C> und <SoftSkill>belastbare Persönlichkeit</SoftSkill>, die ihre Zielsetzung auf überdurchschnittliche Leistungsresultate ausrichtet. == ['Überzeugender Auftritt', 'dynamische Persönlichkeit', 'belastbare Persönlichkeit']",
 " •  <SoftSkill>Freude am Unterrichten</Soft

In [11]:
# train_sjmm_dataset_dict
data_sjmm_dataset_dict = train_sjmm_dataset_dict.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

In [12]:
data_sjmm_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'ind_skill', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 1773
    })
})

### Training

In [13]:
%env WANDB_PROJECT=final_graph

env: WANDB_PROJECT=final_graph


In [14]:

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data['train'],
    train_dataset=data_sjmm_dataset_dict['train'],    
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=1740,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        run_name='t5-3b',
        output_dir='outputs_bloom'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kartikey-sharma (ma-thesis). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.038200
2,2.617100
3,2.576700
4,2.608500
5,2.273400
6,2.415700
7,2.620700
8,2.475400
9,2.638900
10,2.514400


## Share adapters on the 🤗 Hub

In [19]:
huggingface_token = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
model.push_to_hub("Kartikey95/SJMM-bloom-7b1-lora-tagger",
                  use_auth_token=huggingface_token,
                  commit_message="basic training",
                  private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kartikey95/SJMM-bloom-7b1-lora-tagger/commit/7bdf7404181a65a4585c4c9598af52c3a043c6ce', commit_message='basic training', commit_description='', oid='7bdf7404181a65a4585c4c9598af52c3a043c6ce', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [46]:
from huggingface_hub import login
import os

# access_token = os.environ["hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"]
login(token=huggingface_token)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user/ksharma/.cache/huggingface/token
Login successful


In [48]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [49]:
# device_map = {
#     "transformer.word_embeddings": 0,
#     "transformer.word_embeddings_layernorm": 0,
#     "lm_head": "cpu",
#     "transformer.h": 0,
#     "transformer.ln_f": 0,
# }

In [52]:
# model_8bit = AutoModelForCausalLM.from_pretrained(
#     "bigscience/bloom-1b7",
#     device_map=device_map,quantization_config=quantization_config,
# )

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-tagger"

peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-20230716102742"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True,device_map=device_map,quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)




/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"
sjmm_data = pd.read_csv(data_path)
sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
# fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
pd.set_option('display.max_colwidth', 1000)
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
test_sjmm_data

,input,output
752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>."
765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.
1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>."
1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.
1102,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill>wertschätzend miteinader zu kommunizieren</SoftSkill> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>."
...,...,...
306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>."
976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C,<SoftSkill>Proaktives Denken</SoftSkill> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill>vorausschauendes Handeln</SoftSkill>
1412,"<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill","<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill>Ausgeprägte Fähigkeiten in Planung</SoftSkill> und <SoftSkill>Ausgeprägte Fähigkeiten in Organisation</SoftSkill>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill>"
1745,"• ƒ\n<SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill","• <SoftSkill>Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill> un

In [3]:
# batch = tokenizer("<SoftSkill>Unternehmerisches Denken </SoftSkill> und <SoftSkillC>Handeln </SoftSkill_C> == ", return_tensors='pt')
import re
batch = tokenizer("Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. == ", return_tensors='pt')


with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=False)
print('\n\n', generated_text)

match = re.search(r'\[(.*?)\]', generated_text)
extracted_text = match.group(1) if match else None
print('\n\n', extracted_text)


/home/user/ksharma/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. ==  ['dienstleitungsorientierten handel', 'verantwortungsbewussten handel', 'teamorientierten handeln'] Wenn Sie sich erwartenieren, um eine Konzeption zu ver


 'dienstleitungsorientierten handel', 'verantwortungsbewussten handel', 'teamorientierten handeln'


In [28]:
import pandas as pd
import torch

from tqdm import tqdm

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['input', 'expected_text', 'filtered_generated_text', 'actual_generated_text'])

# Iterate over the rows of the input DataFrame with tqdm progress bar
for index, row in tqdm(test_sjmm_data.iterrows(), total=len(test_sjmm_data)):
    # Retrieve the input text from the desired column in each row
    input_text = row['input']
    expected_text = row['ind_skill']

    # Process the input text with the tokenizer to obtain the input tokens
    batch = tokenizer(input_text, return_tensors='pt')

    # Perform inference on the input tokens using your model
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=80)

    # Decode the output tokens to obtain the generated text
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    match = re.search(r'\[(.*?)\]', generated_text)
    extracted_text = match.group(1) if match else None

    # print('\n\n', extracted_text)

    # Store the input text and generated text in the results DataFrame
    results_df = results_df.append({'input': input_text, 'expected_text': expected_text,'filtered_generated_text':extracted_text, 'actual_generated_text': generated_text}, ignore_index=True)




100%|███████████████████████████████████████████████████████████████████████████████████| 198/198 [27:42<00:00,  8.40s/it]


In [29]:
results_df.tail(5)

,input,expected_text,filtered_generated_text,actual_generated_text
193,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","[agieren in schwierigen Situationen ruhig, agieren vertrauenswürdig, agieren mit souveräner Umsicht]","'agieren in schwierigen Situationen ruhig', 'agieren in schwierigen Situationen vertrauenswürdig', 'agieren in schwierigen Situationen mit souveräner Umsicht'","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>. == ['agieren in schwierigen Situationen ruhig', 'agieren in schwierigen Situationen vertrauenswürdig', 'agieren in schwierigen Situationen mit souveräner Umsicht'] == ['agieren in schwierigen Situationen ruhig', 'agieren in schwierigen Situationen vertrauens"
194,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C,"[Proaktives Denken, vorausschauendes Denken, vorausschauendes Handeln]","'Proaktives Denken', 'vorausschauendes Denken', 'proaktives Handeln'","• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C == ['Proaktives Denken', 'vorausschauendes Denken', 'proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Handeln'] == ['proaktives Hand"
195,"<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill","[Ausgeprägte Fähigkeiten in Konzeption, Analyse, Ausgeprägte Fähigkeiten in Planung, Ausgeprägte Fähigkeiten in Organisation, vernetztes Denken, kritisches Hinterfragen]","'Ausgeprägte Fähigkeiten in Konzeption, Analyse', 'Ausgeprägte Fähigkeiten in Planung', 'Ausgeprägte Fähigkeiten in Organisation', 'vernetztes Denken', 'kritisches Hinterfragen'","<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill>. == ['Ausgeprägte Fähigkeiten in Konzeption, Analyse', 'Ausgeprägte Fähigkeiten in Planung', 'Ausgeprägte Fähigkeiten in Organisation', 'vernetztes Denken', 'kritisches Hinterfragen'] == ['Ausgep"
196,"• ƒ\n<SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill","[Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen, Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen]","'Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen', 'Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen'","• ƒ\n<SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill == ['Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen', 'Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen'] == ['Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen', 'Fähigkeit, sich fachkompetent in der Projektarbeit zu"
197,"B und Hubstapler\r\n<SoftSkill>Prozessorientierte Denkweise</SoftSkill> und <SoftSkill>sehr gute IT-Affinität</SoftSkill>\r\n<SoftSkill_C>Selbständiges</SoftSkill_C> und <SoftSkill>sorgfältiges Arbeiten</SoftSkill> mit <SoftSkill>Eigeninitiative</SoftSkill>\r\n<SoftSkill>Bereitschaft, Samstag/Sonntag sowie an Feiertagen Arbeitseinsätze zu leisten</SoftSkill","[Prozessorientierte Denkweise, sehr gute IT-Affinität, Selbständiges Arbeiten, sorgfältiges Arbeiten, Eigeninitiative, Bereitschaft, Samstag/Sonntag sowie an Feiertagen Arbeitse

In [30]:
results_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_15epochs.csv")

: 

In [23]:
# pd.read_csv('bloom_7bn_results_15epochs.csv')

,Unnamed: 0,input,expected_text,filtered_generated_text,actual_generated_text
0,0,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']","'dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. == ['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln] == ['dienstleitungsorientierten handel"
1,1,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']","'Arbeitsweise ist proaktiv', 'ihre Arbeitsweise ist hoher Dienstleistungsbereitschaft', 'ihre Arbeitsweise ist hoher Einsatzbereitschaft'","Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>. == ['Arbeitsweise ist proaktiv', 'ihre Arbeitsweise ist hoher Dienstleistungsbereitschaft', 'ihre Arbeitsweise ist hoher Einsatzbereitschaft'] == =="
2,2,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",NaN,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>. == ['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikations-Fähigkeit"
3,3,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,"['wertschätzen die Erfahrung der Betroffenen und deren Angehörigen', 'respektieren die Erfahrung der Betroffenen und deren Angehörigen']","'wertschätzen die Erfahrung der Betroffenen und deren Angehörigen', 'respektieren die Erfahrung der Betroffenen und deren Angehörigen'","Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>. == ['wertschätzen die Erfahrung der Betroffenen und deren Angehörigen', 'respektieren die Erfahrung der Betroffenen und deren Angehörigen'] == ['"
4,4,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","['wertschätzend miteinader zu kommunizieren', 'respektvoll miteinader zu kommunizieren', 'konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten']",NaN,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <

## Model adapters save to local machine 

In [31]:
# model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
# model_to_save.save_pretrained("outputs_bloom")

## Model Load from local 

In [32]:
# from peft import get_peft_model
# lora_config = LoraConfig.from_pretrained('outputs_bloom')
# model = get_peft_model(model, lora_config)

In [34]:

# batch = tokenizer("• <SoftSkill_C>Motivations-</SoftSkill_C> und <SoftSkill>Begeisterungsfähigkeit</SoftSkill", return_tensors='pt')


# with torch.cuda.amp.autocast():
#   output_tokens = model.generate(**batch, max_new_tokens=50)

# print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 • <SoftSkill_C>Motivations-</SoftSkill_C> und <SoftSkill>Begeisterungsfähigkeit</SoftSkill>.</SoftSkill>
 *
 * @author Florian Allerding
 */
public class SoftSkill {

    private String name;
    private String description;
    private String value;
    private String type;
    private String category;
    private String level;


## Testing for ORIG approach

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-tagger"
# 
peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-ORIG-stopseq20230717114608"
# peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-ORIG-20230717080523"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True,device_map=device_map,quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)



/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# batch = tokenizer("<SoftSkill>Unternehmerisches Denken </SoftSkill> und <SoftSkillC>Handeln </SoftSkill_C> == ", return_tensors='pt')
import re
batch = tokenizer("Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>. == ", return_tensors='pt')


with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=280)

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=False, stop = " ###")
# print('\n\n', generated_text)

start_index = generated_text.find("==") + 3
end_index = generated_text.find(" ###")

extracted_text = generated_text[start_index:end_index]

print(extracted_text)

# match = re.search(r'\[(.*?)\]', generated_text)
# extracted_text = match.group(1) if match else None
# print('\n\n', extracted_text)

/home/user/ksharma/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


 Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfreudigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.


In [5]:
import pandas as pd 
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/MASTER_TEST_SET.csv"
test_set = pd.read_csv(data_path)
test_set = test_set[test_set['valid'] == 1]
# test_set['expected_skills'] = test_set['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
test_set

,orig_idx,input,expected_text,Problem type,expected_skills,valid
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1
3,1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,MWA,"['wertschätzen die Erfahrung der Betroffenen und deren Angehörigen', 'respektieren die Erfahrung der Betroffenen und deren Angehörigen']",1
4,1102,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill>wertschätzend miteinader zu kommunizieren</SoftSkill> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.",MWA,"['wertschätzend miteinader zu kommunizieren', 'respektvoll miteinader zu kommunizieren', 'konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten']",1
...,...,...,...,...,...,...
193,306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>.",MWA,"['agieren in schwierigen Situationen ruhig', 'agieren vertrauenswürdig', 'agieren mit souveräner Umsicht']",1
194,976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</So

In [4]:
test_sjmm_data

,input,output
752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>."
765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.
1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>."
1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.
1102,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill>wertschätzend miteinader zu kommunizieren</SoftSkill> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>."
...,...,...
306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>."
976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C,<SoftSkill>Proaktives Denken</SoftSkill> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill>vorausschauendes Handeln</SoftSkill>
1412,"<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill","<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill>Ausgeprägte Fähigkeiten in Planung</SoftSkill> und <SoftSkill>Ausgeprägte Fähigkeiten in Organisation</SoftSkill>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill>"
1745,"• ƒ\n<SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill","• <SoftSkill>Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill> un

In [5]:
# start_index = generated_text.find("==") + 3
# end_index = generated_text.find(" ###")

# extracted_text = generated_text[start_index:end_index]

# print(extracted_text)

 Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.


In [6]:
test_set.columns

Index(['orig_idx', 'input', 'expected_text', 'Problem type', 'expected_skills',
       'valid'],
      dtype='object')

In [8]:
import pandas as pd
import torch

from tqdm import tqdm

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['orig_idx', 'input', 'expected_text', 'generated_text', 'expected_skills','generated_skills' ])

# Iterate over the rows of the input DataFrame with tqdm progress bar
for index, row in tqdm(test_set.iterrows(), total=len(test_set)):
    # Retrieve the input text from the desired column in each row
    input_text = row['input']
    expected_text = row['expected_text']
    orig_idx = row['orig_idx']
    problem_type = row['Problem type']
    valid = row['valid']
    expected_skills = row['expected_skills']

    # Process the input text with the tokenizer to obtain the input tokens
    batch = tokenizer(input_text, return_tensors='pt')

    # Perform inference on the input tokens using your model
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=280)

    # Decode the output tokens to obtain the generated text
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    start_index = generated_text.find("==") + 3
    end_index = generated_text.find(" ###")

    extracted_text = generated_text[start_index:end_index]
    gen_skills = re.findall(r'(?:<SoftSkill>|SoftSkill_C>)(.*?)(?:</SoftSkill>|</SoftSkill_C>)', extracted_text)
    # results_bloom['generated_skills'] = results_bloom['generated_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')

    # Store the input text and generated text in the results DataFrame
    results_df = results_df.append({'input': input_text, 'expected_text': expected_text,'generated_text':extracted_text, 'expected_skills': expected_skills, 'generated_skills': gen_skills, 'orig_idx': orig_idx, 'Problem type': problem_type, 'valid': valid}, ignore_index=True)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195/195 [1:35:41<00:00, 29.44s/it]


In [12]:
results_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")

In [9]:
results_df

,orig_idx,input,expected_text,generated_text,expected_skills,generated_skills,Problem type,valid
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]",MC,1.0
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",HOWA,1.0
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfreudigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfreudigkeit, Kommunikations-Fähigkeit, praktisches Verständnis, manuelles Geschick]",HMDC,1.0
3,1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,"['wertschätzen die Erfahrung der Betroffenen und deren Angehörigen', 'respektieren die Erfahrung der Betroffenen